<font color=pink size=100>环境准备+代理</font>


In [1]:
# 导入必要的库并打印版本信息
import langchain
import langchain_community
import pypdf
import sentence_transformers
print(1)
for module in (langchain, langchain_community, pypdf, sentence_transformers):
    print(f"{module.__name__:<30}{module.__version__}")

#导入操作系统和数据处理相关库
import os
import pandas as pd


# 定义嵌入模型
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
import torch
from langchain_ollama import OllamaLLM
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')
#定义嵌入模型，跑通这部分代码需要开代理
embedding_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-large-zh-v1.5',
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True}
)

#加载向量数据库
# 加载 index.faiss 文件作为向量数据库
from langchain_community.vectorstores.faiss import FAISS
import faiss
import os
import numpy as np
# 确保路径正确数据库文件\index.faiss
index_path = r"E:\RAG\database"
if os.path.exists(index_path):
    print("index.faiss 文件存在")
    vector_db = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True)
    
# 输出向量数据库vector_db中包含的数据条数
print(f"向量数据库中数据总数: {vector_db.index.ntotal}")

# #采用ollama的LLM模型千问
# llm = OllamaLLM(model="qwen:7b")

1
langchain                     0.3.25
langchain_community           0.3.25
pypdf                         5.6.0
sentence_transformers         4.1.0
device: cpu
device: cpu
index.faiss 文件存在
向量数据库中数据总数: 1163
index.faiss 文件存在
向量数据库中数据总数: 1163


<font color=pink size=50>简单检索</font>

In [2]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
from rank_bm25 import BM25Okapi
import os
import re

#加载QA1，并提取页码信息
qa_file_path = '../QA对/QA4.json'
with open(qa_file_path, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

answers = [item['answer'] for item in qa_data]
pages = [item['page_num'] for item in qa_data]  
queries = [item['question'] for item in qa_data]
k = 50 #返回检索结果的数量

def jieba_tokenizer(text):
    return list(jieba.cut(text))

# 定义简单的RAG检索函数
def rag_search(query, k=k):
    """
    使用vector_db对输入query进行检索，返回前k条结果。
    """
    retriever = vector_db.as_retriever(search_kwargs={"k": k})
    results = retriever.invoke(query)
    return results

output_dir = '../outputs/outputs_4.0/简单检索'
os.makedirs(output_dir, exist_ok=True)
output_file_path = os.path.join(output_dir, f'rag_results_easy_k={k}.json')

recall_list = []
output_data = []
mrr_list = []

# 新增：对每个问题进行检索，输出内容和页码
for idx, (query, answer, correct_page) in enumerate(zip(queries, answers, pages)):
    results = rag_search(query, k=k)
    retrieved_documents = [
        {
            "uuid":doc.metadata.get("uuid", "未指定"),
            "source_file": doc.metadata.get("source_file", ""),
            "source_content": doc.page_content,
            "page_num": str(doc.metadata.get("page_num", "未指定"))
        }
        for doc in results
    ]
    retrieved_pages = [doc["page_num"] for doc in retrieved_documents]
    # 检查是否有检索到的页码与标准答案页码一致，若有recall=1，否则recall=0
    recall = 1 if correct_page in retrieved_pages else 0
    recall_list.append(recall)
    # 计算MRR，即第一个正确答案的倒数排名
    rank = 0
    for i, p in enumerate(retrieved_pages):
        if p == correct_page:
            rank = i + 1
            break
    mrr = 1.0 / rank if rank > 0 else 0.0
    mrr_list.append(mrr)
    output_data.append({
        "query": query,
        "answer": answer,
        "correct_page": correct_page,
        "recall": recall,
        "mrr": mrr,
        "retrieved_documents": retrieved_documents
    })

avg_recall = sum(recall_list) / len(recall_list) if recall_list else 0.0
avg_mrr = sum(mrr_list) / len(mrr_list) if mrr_list else 0.0

# 保存整体指标到输出结果
summary = {
    "avg_recall": avg_recall,
    "avg_mrr": avg_mrr
}
output = {
    "summary": summary,
    "results": output_data
}
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=False, indent=4)

print(f"检索结果已保存到 {output_file_path}")

检索结果已保存到 ../outputs/outputs_4.0/简单检索\rag_results_easy_k=50.json


<font color=pink size=20>混合检索</font>

In [ ]:
import numpy as np
from rank_bm25 import BM25Okapi
import jieba
import json
import os
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import cpu_count
from functools import partial
from sklearn.metrics.pairwise import cosine_similarity
k=50  # 返回检索结果的数量
def tokenize(text):
    """使用jieba进行中文分词"""
    return list(jieba.cut(text))

def create_bm25_index(documents):
    """创建BM25索引"""
    tokenized_docs = [tokenize(doc.page_content) for doc in documents]
    return BM25Okapi(tokenized_docs)

def hybrid_search(query, bm25_index, documents, embeddings, embedding_model, k=20, alpha=0.75):
    """混合检索实现"""
    # BM25检索
    tokenized_query = tokenize(query)
    bm25_scores = bm25_index.get_scores(tokenized_query)
    
    # Embedding检索
    query_embedding = embedding_model.embed_query(query)
    # 确保embeddings是numpy数组
    embeddings = np.array(embeddings)
    query_embedding = np.array(query_embedding).reshape(1, -1)
    
    # 计算余弦相似度
    embedding_scores = cosine_similarity(query_embedding, embeddings)[0]
    
    # 归一化分数
    bm25_scores = (bm25_scores - bm25_scores.min()) / (bm25_scores.max() - bm25_scores.min() + 1e-6)
    embedding_scores = (embedding_scores - embedding_scores.min()) / (embedding_scores.max() - embedding_scores.min() + 1e-6)
    
    # 混合分数
    hybrid_scores = alpha * bm25_scores + (1 - alpha) * embedding_scores
    
    # 获取top-k结果
    top_k_indices = np.argsort(hybrid_scores)[-k:][::-1]
    
    # 构建结果
    results = []
    for idx in top_k_indices:
        doc = documents[idx]
        results.append({
            "uuid": doc.metadata.get("uuid", "未指定"),
            "source_file": doc.metadata.get("source_file", ""),
            "source_content": doc.page_content,
            "page_num": str(doc.metadata.get("page_num", "未指定")),
            "score": float(hybrid_scores[idx]),
            "bm25_score": float(bm25_scores[idx]),
            "embedding_score": float(embedding_scores[idx])
        })
    return results

def process_hybrid_search(query, bm25_index, documents, embeddings, embedding_model, k=20):
    """处理单个查询的混合检索"""
    try:
        results = hybrid_search(
            query=query,
            bm25_index=bm25_index,
            documents=documents,
            embeddings=embeddings,
            embedding_model=embedding_model,
            k=k
        )
        return {
            'query': query,
            'retrieved_documents': results
        }
    except Exception as e:
        print(f"处理查询时出错: {str(e)}")
        return {
            'query': query,
            'retrieved_documents': [],
            'error': str(e)
        }

def parallel_hybrid_search(queries, documents, embeddings, embedding_model, k=k):
    """并行处理多个查询的混合检索"""
    # 创建BM25索引
    bm25_index = create_bm25_index(documents)
    
    # 确保embeddings是numpy数组
    embeddings = np.array(embeddings)
    
    # 使用线程池并行处理所有查询
    with ThreadPoolExecutor(max_workers=cpu_count()) as executor:
        # 使用partial固定参数
        process_func = partial(
            process_hybrid_search,
            bm25_index=bm25_index,
            documents=documents,
            embeddings=embeddings,
            embedding_model=embedding_model,
            k=k
        )
        results = list(executor.map(process_func, queries))
    
    return results

def main():
    # 加载QA4.json中的问题
    qa_file_path = '../QA对/QA4.json'
    with open(qa_file_path, 'r', encoding='utf-8') as f:
        qa_data = json.load(f)
    
    # 提取问题和答案
    queries = [item['question'] for item in qa_data]
    answers = [item['answer'] for item in qa_data]
    pages = [item['page_num'] for item in qa_data]
    
    # 获取所有文档
    all_docs = list(vector_db.docstore._dict.values())
    
    # 获取所有文档的embeddings
    all_texts = [doc.page_content for doc in all_docs]
    embeddings = embedding_model.embed_documents(all_texts)
    
    # 执行混合检索
    results = parallel_hybrid_search(
        queries=queries,
        documents=all_docs,
        embeddings=embeddings,
        embedding_model=embedding_model,
        k=k
    )
    
    # 计算评估指标
    recall_list = []
    mrr_list = []
    output_data = []
    
    for idx, (result, answer, correct_page) in enumerate(zip(results, answers, pages)):
        retrieved_pages = [doc["page_num"] for doc in result['retrieved_documents']]
        
        # 计算recall
        recall = 1 if correct_page in retrieved_pages else 0
        recall_list.append(recall)
        
        # 计算MRR
        rank = 0
        for i, p in enumerate(retrieved_pages):
            if p == correct_page:
                rank = i + 1
                break
        mrr = 1.0 / rank if rank > 0 else 0.0
        mrr_list.append(mrr)
        
        # 构建输出数据
        output_data.append({
            "query": result['query'],
            "answer": answer,
            "correct_page": correct_page,
            "recall": recall,
            "mrr": mrr,
            "retrieved_documents": result['retrieved_documents']
        })
    
    # 计算平均指标
    avg_recall = sum(recall_list) / len(recall_list) if recall_list else 0.0
    avg_mrr = sum(mrr_list) / len(mrr_list) if mrr_list else 0.0
    
    # 保存结果
    output_dir = '../outputs/outputs_4.0/混合检索'
    os.makedirs(output_dir, exist_ok=True)
    output_file_path = os.path.join(output_dir, f'hybrid_results_alpha=0.75_k={k}.json')
    
    output = {
        "summary": {
            "avg_recall": avg_recall,
            "avg_mrr": avg_mrr
        },
        "results": output_data
    }
    
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(output, f, ensure_ascii=False, indent=4)
    
    print(f"混合检索结果已保存到 {output_file_path}")
    print(f"平均Recall: {avg_recall:.4f}")
    print(f"平均MRR: {avg_mrr:.4f}")

if __name__ == "__main__":
    main()

<font color=pink size=20 >多查询检索+混合检索</font>

In [ ]:
#使用多套提示词，添加MRR和Recall指标，生成输出文件.json
from typing import List
from langchain_core.output_parsers import BaseOutputParser
import concurrent.futures
import requests
import json
import os
import time
from typing import Tuple, Dict, List, Any
from typing import List, Dict
from langchain.prompts import PromptTemplate
k = 50

# 定义提示词模板
PROMPT_TEMPLATE = """您是查询扩展方面的专家，能够生成问题的释义。
我无法直接使用用户的问题从知识库中检索相关信息。
您需要通过多种方式扩展或释义用户的问题，例如使用同义词/短语、完整地写出缩写、添加一些额外的描述或解释、改变表达方式、将原始问题翻译成中文等。
并返回 5 个版本的问题，其中一个来自翻译。
只需列出问题。不需要其他单词。

原始问题：{query}"""


class LineListOutputParser(BaseOutputParser[List[str]]):
    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))

output_parser = LineListOutputParser()

API_KEY = "3757050b-82ca-41c0-a872-065b7913ff87"
MODEL_NAME = "doubao-1.5-thinking-pro-m-250428"

import os
from volcenginesdkarkruntime import Ark

# 创建方舟客户端
client = Ark(
    api_key=API_KEY,  # 使用已定义的 API_KEY
    timeout=1800,     # 设置30分钟超时
)

def call_llm_api(prompt: str, max_retries: int = 3) -> str:
    """调用LLM API生成回答"""
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "你是一个专业的保险问题生成助手。"},
                    {"role": "user", "content": prompt}
                ]
            )
            return response.choices[0].message.content
        except Exception as e:
            if attempt == max_retries - 1:
                raise Exception(f"API调用失败: {str(e)}")
            time.sleep(2 ** attempt)  # 指数退避


def calculate_metrics(documents, correct_page):
    """计算召回率和MRR"""
    retrieved_pages = [str(doc.metadata.get("page_num", "")) for doc in documents]
    recall = 1 if correct_page in retrieved_pages else 0
    
    rank = 0
    for i, p in enumerate(retrieved_pages):
        if p == correct_page:
            rank = i + 1
            break
    mrr = 1.0 / rank if rank > 0 else 0.0
    
    return recall, mrr


def retrieve_documents(question: str, retriever) -> List[Any]:
    """单个问题的检索函数"""
    try:
        # 使用混合检索，同时考虑关键词匹配和语义相似度
        docs = retriever.get_relevant_documents(
            question,
            search_type="hybrid",  # 使用混合检索
            search_kwargs={
                "k": k,
                "fetch_k": k * 2,  # 获取更多候选文档
                "lambda_mult": 0.5,  # 平衡关键词和语义相似度的权重
                "hybrid_search_type": "both"  # 同时使用关键词和语义搜索
            }
        )
        return docs
    except Exception as e:
        print(f"检索出错: {str(e)}")
        return []

def process_query(query: str, correct_page: str, answer: str, retriever) -> Tuple[Dict, float, float]:
    """处理单个查询"""
    try:
        # 使用提示词模板生成问题
        prompt = PROMPT_TEMPLATE.format(query=query)
        generated_questions = call_llm_api(prompt).strip().split('\n')
        
        # 并行检索所有生成的问题
        raw_documents = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(5, len(generated_questions))) as executor:
            # 创建检索任务
            future_to_question = {
                executor.submit(retrieve_documents, question, retriever): question 
                for question in generated_questions
            }
            
            # 收集检索结果
            for future in concurrent.futures.as_completed(future_to_question):
                docs = future.result()
                raw_documents.extend(docs)
        
        # 文档去重
        seen_contents = set()
        unique_documents = []
        for doc in raw_documents:
            content = doc.page_content.strip()
            if content not in seen_contents:
                seen_contents.add(content)
                unique_documents.append(doc)
        
        # 计算召回率和MRR
        recall, mrr = calculate_metrics(unique_documents, correct_page)
         # 将 Document 对象转换为可序列化的字典
        serializable_docs = []
        for doc in unique_documents:
            serializable_docs.append({
                "page_content": doc.page_content,
                "metadata": doc.metadata
            })
        
        return {
            "query": query,
            "correct_page": correct_page,
            "answer": answer,  # 添加标准答案
            "recall": recall,
            "mrr": mrr,
            "generated_questions": generated_questions,
            "retrieved_documents": serializable_docs  # 使用可序列化的文档列表
        }, recall, mrr
    except Exception as e:
        print(f"处理查询时出错: {str(e)}")
        return {
            "query": query,
            "correct_page": correct_page,
            "answer": answer,  # 添加标准答案
            "recall": 0,
            "mrr": 0,
            "generated_questions": [],
            "retrieved_documents": [],
            "error": str(e)
        }, 0, 0

def evaluate_queries(queries, correct_pages, answers, retriever):
    """评估查询结果"""
    results = []
    total_recall = 0
    total_mrr = 0
    
    for query, correct_page, answer in zip(queries, correct_pages, answers):
        result, recall, mrr = process_query(query, correct_page, answer, retriever)
        results.append(result)
        total_recall += recall
        total_mrr += mrr
    
    avg_recall = total_recall / len(queries)
    avg_mrr = total_mrr / len(queries)
    
    return results, avg_recall, avg_mrr

# 执行评估
# 创建检索器
# 加载QA数据
qa_data_path =  '../QA对/QA4.json'
with open(qa_data_path, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

# 提取查询、答案和正确页码
queries = [item['question'] for item in qa_data]
answers = [item['answer'] for item in qa_data]
pages = [item['page_num'] for item in qa_data]

# 执行评估
retriever = vector_db.as_retriever(search_kwargs={"k": k})
results, avg_recall, avg_mrr = evaluate_queries(queries, pages, answers, retriever)

# 保存结果
output = {
    "summary": {
        "avg_recall": avg_recall,
        "avg_mrr": avg_mrr,
        "total_questions": len(qa_data)
    },
    "results": results
}

# 确保输出目录存在
output_dir = '../outputs/outputs_4.0/多提示词检索'
os.makedirs(output_dir, exist_ok=True)

# 保存结果到文件
output_file_path = os.path.join(output_dir, f'multi_prompt_results_k={k}.json')
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print(f"评估完成！结果已保存到: {output_file_path}")
print(f"平均Recall: {avg_recall:.4f}")
print(f"平均MRR: {avg_mrr:.4f}")

In [ ]:
%pip install volcenginesdk-ark-runtime

import numpy as np
from rank_bm25 import BM25Okapi
import jieba
import json
import os
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import cpu_count
from functools import partial
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict
from volcenginesdkarkruntime import Ark
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import BaseOutputParser

# 配置
API_KEY = "3757050b-82ca-41c0-a872-065b7913ff87"
MODEL_NAME = "deepseek-r1-distill-qwen-32b-250120"
k = 50  # 返回检索结果的数量

# 创建方舟客户端
client = Ark(
    api_key=API_KEY,
    timeout=1800,
)

# 定义提示词模板
PROMPT_TEMPLATE = """您是查询扩展方面的专家，能够生成问题的释义。
我无法直接使用用户的问题从知识库中检索相关信息。
您需要通过多种方式扩展或释义用户的问题，例如使用同义词/短语、完整地写出缩写、添加一些额外的描述或解释、改变表达方式、将原始问题翻译成中文等。
并返回 5 个版本的问题，其中一个来自翻译。
只需列出问题。不需要其他单词。

原始问题：{query}"""

class LineListOutputParser(BaseOutputParser[List[str]]):
    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))

output_parser = LineListOutputParser()

def tokenize(text):
    """使用jieba进行中文分词"""
    return list(jieba.cut(text))

def create_bm25_index(documents):
    """创建BM25索引"""
    tokenized_docs = [tokenize(doc.page_content) for doc in documents]
    return BM25Okapi(tokenized_docs)

def llm_rerank_api(query: str, results: List[Dict], top_k: int = k) -> List[Dict]:
    """使用API调用LLM对检索结果进行重排序"""
    try:
        # 构建重排序提示词
        rerank_prompt = f"""你是一个专业的文档相关性评估专家。请评估以下文档与问题的相关性，并给出0-1之间的分数。
问题：{query}

请对每个文档进行评分，分数越高表示与问题越相关。评分标准：
1. 文档内容是否直接回答了问题
2. 文档内容与问题的语义相关性
3. 文档内容的完整性和准确性

文档列表：
"""
        
        # 为每个文档添加编号和内容
        for i, doc in enumerate(results):
            rerank_prompt += f"\n文档{i+1}：\n{doc['source_content']}\n"
        
        rerank_prompt += "\n请给出每个文档的评分（0-1之间的小数），格式为：文档编号:分数"
        
        # 调用API进行重排序
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "你是一个专业的文档相关性评估专家。"},
                {"role": "user", "content": rerank_prompt}
            ]
        )
        
        # 解析API返回的评分
        scores_text = response.choices[0].message.content
        scores = {}
        for line in scores_text.split('\n'):
            if ':' in line:
                doc_num, score = line.split(':')
                try:
                    doc_num = int(doc_num.strip().replace('文档', ''))
                    score = float(score.strip())
                    scores[doc_num] = score
                except:
                    continue
        
        # 将LLM评分添加到结果中
        for i, doc in enumerate(results):
            doc['llm_score'] = scores.get(i+1, 0.0)
        
        # 根据LLM评分重排序
        reranked_results = sorted(results, key=lambda x: x['llm_score'], reverse=True)
        
        return reranked_results[:top_k]
    except Exception as e:
        print(f"LLM重排序出错: {str(e)}")
        return results[:top_k]  # 如果重排序失败，返回原始排序结果

def hybrid_search(query, bm25_index, documents, embeddings, embedding_model, k=k, alpha=0.5):
    """混合检索实现"""
    # BM25检索
    tokenized_query = tokenize(query)
    bm25_scores = bm25_index.get_scores(tokenized_query)
    
    # Embedding检索
    query_embedding = embedding_model.embed_query(query)
    # 确保embeddings是numpy数组
    embeddings = np.array(embeddings)
    query_embedding = np.array(query_embedding).reshape(1, -1)
    
    # 计算余弦相似度
    embedding_scores = cosine_similarity(query_embedding, embeddings)[0]
    
    # 归一化分数
    bm25_scores = (bm25_scores - bm25_scores.min()) / (bm25_scores.max() - bm25_scores.min() + 1e-6)
    embedding_scores = (embedding_scores - embedding_scores.min()) / (embedding_scores.max() - embedding_scores.min() + 1e-6)
    
    # 混合分数
    hybrid_scores = alpha * bm25_scores + (1 - alpha) * embedding_scores
    
    # 获取top-k结果
    top_k_indices = np.argsort(hybrid_scores)[-k:][::-1]
    
    # 构建结果
    results = []
    for idx in top_k_indices:
        doc = documents[idx]
        results.append({
            "uuid": doc.metadata.get("uuid", "未指定"),
            "source_file": doc.metadata.get("source_file", ""),
            "source_content": doc.page_content,
            "page_num": str(doc.metadata.get("page_num", "未指定")),
            "score": float(hybrid_scores[idx]),
            "bm25_score": float(bm25_scores[idx]),
            "embedding_score": float(embedding_scores[idx])
        })
    
    return results

def multi_prompt_retrieve(query, bm25_index, documents, embeddings, embedding_model, k=k):
    """多查询检索实现"""
    # 使用固定的prompt模板进行查询扩展
    prompt = PromptTemplate(input_variables=["query"], template=PROMPT_TEMPLATE)
    llm_chain = prompt | llm | output_parser
    expanded_queries = llm_chain.invoke(query)
    
    # 对每个扩展问题分别进行混合检索，合并去重
    doc_set = {}
    for q in expanded_queries:
        # 使用hybrid_search替代hybrid_retrieve
        results = hybrid_search(q, bm25_index, documents, embeddings, embedding_model, k=k)
        for doc in results:
            doc_id = doc["uuid"] + doc["page_num"]
            if doc_id not in doc_set:
                doc_set[doc_id] = doc
    
    # 将结果转换为列表
    results = list(doc_set.values())
    
    # 使用LLM进行重排序
    reranked_results = llm_rerank_api(query, results, top_k=k)
    
    return reranked_results

def main():
    # 加载QA4.json中的问题
    qa_file_path = '../QA对/QA4.json'
    with open(qa_file_path, 'r', encoding='utf-8') as f:
        qa_data = json.load(f)
    
    # 提取问题和答案
    queries = [item['question'] for item in qa_data]
    answers = [item['answer'] for item in qa_data]
    pages = [item['page_num'] for item in qa_data]
    
    # 获取所有文档
    all_docs = list(vector_db.docstore._dict.values())
    
    # 创建BM25索引
    bm25_index = create_bm25_index(all_docs)
    
    # 获取所有文档的embeddings
    all_texts = [doc.page_content for doc in all_docs]
    embeddings = embedding_model.embed_documents(all_texts)
    
    # 执行多查询混合检索
    results = []
    for query in queries:
        result = multi_prompt_retrieve(
            query, 
            bm25_index, 
            all_docs, 
            embeddings, 
            embedding_model, 
            k=k
        )
        results.append({
            'query': query,
            'retrieved_documents': result
        })
    
    # 计算评估指标
    recall_list = []
    mrr_list = []
    output_data = []
    
    for idx, (result, answer, correct_page) in enumerate(zip(results, answers, pages)):
        retrieved_pages = [doc["page_num"] for doc in result['retrieved_documents']]
        
        # 计算recall
        recall = 1 if correct_page in retrieved_pages else 0
        recall_list.append(recall)
        
        # 计算MRR
        rank = 0
        for i, p in enumerate(retrieved_pages):
            if p == correct_page:
                rank = i + 1
                break
        mrr = 1.0 / rank if rank > 0 else 0.0
        mrr_list.append(mrr)
        
        # 构建输出数据
        output_data.append({
            "query": result['query'],
            "answer": answer,
            "correct_page": correct_page,
            "recall": recall,
            "mrr": mrr,
            "retrieved_documents": result['retrieved_documents']
        })
    
    # 计算平均指标
    avg_recall = sum(recall_list) / len(recall_list) if recall_list else 0.0
    avg_mrr = sum(mrr_list) / len(mrr_list) if mrr_list else 0.0
    
    # 保存结果
    output_dir = '../outputs/outputs_4.0/多查询_混合检索_LLM重排序'
    os.makedirs(output_dir, exist_ok=True)
    output_file_path = os.path.join(output_dir, f'multi_hybrid_llm_rerank_results_k={k}.json')
    
    output = {
        "summary": {
            "avg_recall": avg_recall,
            "avg_mrr": avg_mrr
        },
        "results": output_data
    }
    
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(output, f, ensure_ascii=False, indent=4)
    
    print(f"多查询混合检索+LLM重排序结果已保存到 {output_file_path}")
    print(f"平均Recall: {avg_recall:.4f}")
    print(f"平均MRR: {avg_mrr:.4f}")

if __name__ == "__main__":
    main()

<font color=pink size=20 >上下文压缩</font>

In [ ]:
#使用API方式压缩文档
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document 
import json
import os
from multiprocessing import Pool, cpu_count
from functools import partial
import concurrent.futures
import random
import requests
API_KEY = "3757050b-82ca-41c0-a872-065b7913ff87"
API_BASE = "https://ark.cn-beijing.volces.com/api/v3"
MODEL_NAME = "deepseek-r1-250120"
def compress_with_api(doc, query, api_key, api_base, model_name):
    """
    使用API调用大语言模型对单个文档进行压缩。
    """
    prompt = (
        f"请根据以下问题和文档内容，提取与问题最相关的内容，去除无关信息，输出精炼后的内容。\n"
        f"问题：{query}\n"
        f"文档内容：{doc.page_content}\n"
        f"压缩后内容："
    )
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "你是一个文档压缩助手，善于根据问题压缩文档内容。"},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0,
        "max_tokens": 1024
    }
    response = requests.post(
        f"{API_BASE}/chat/completions",
        headers=headers,
        json=payload,
        timeout=60
    )
    if response.status_code == 200:
        result = response.json()
        return result["choices"][0]["message"]["content"].strip()
    else:
        print(f"API调用失败: {response.status_code}, {response.text}")
        return doc.page_content  # 失败时返回原文

def compress_document(doc, query):
    """API方式压缩单个文档"""
    try:
        compressed_content = compress_with_api(
            doc, query,
            api_key=API_KEY,
            api_base=API_BASE,
            model_name=MODEL_NAME
        )
        return Document(
            page_content=compressed_content,
            metadata=doc.metadata
        )
    except Exception as e:
        print(f"API压缩文档时出错: {str(e)}")
        return doc

def parallel_compress_documents(docs, query, num_threads=8):
    """并行压缩多个文档（API方式，线程池）"""
    import concurrent.futures
    if len(docs) > 10:
        docs = random.sample(docs, 10)
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(compress_document, doc, query) for doc in docs]
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())
    return results

def compress_one(item):
    """压缩单个查询的所有文档"""
    query = item['query']
    docs = []
    for doc_dict in item['retrieved_documents']:
        doc = Document(
            page_content=doc_dict.get('content', ''),
            metadata={
                "uuid": doc_dict.get("uuid", "未指定"),
                "source_file": doc_dict.get("source_file", ""),
                "page": doc_dict.get("page", "未指定")
            }
        )
        docs.append(doc)
    compressed_docs = parallel_compress_documents(docs, query)
    for i, doc_dict in enumerate(item['retrieved_documents']):
        if i < len(compressed_docs):
            doc_dict['compressed_content'] = compressed_docs[i].page_content
    return item

def process_compression(input_file_path, output_file_path, k=20):
    """主处理函数（API方式）"""
    with open(input_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    import concurrent.futures
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        results = list(executor.map(compress_one, data['results']))
    data['results'] = results
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"压缩结果已保存到 {output_file_path}")

if __name__ == "__main__":
    # 设置输入输出路径
    input_file_path = f'../outputs/outputs_4.0/混合检索/hybrid_results_k={k}.json'
    output_dir = '../outputs/outputs_4.0/上下文压缩'
    os.makedirs(output_dir, exist_ok=True)
    output_file_path = os.path.join(output_dir, f'compressed_results_k={k}.json')
    process_compression(
        input_file_path=input_file_path,
        output_file_path=output_file_path,
        k=k
    )

<font color=pink size=20 >添加问答链</font>

In [ ]:
import os
import json
import random
from typing import List, Dict, Tuple
from volcenginesdkarkruntime import Ark # 方舟API客户端
from nltk.translate.bleu_score import sentence_bleu # BLEU评分
from rouge_chinese import Rouge # 中文ROUGE评分
import jieba # 中文分词
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# 配置
API_KEY = "3757050b-82ca-41c0-a872-065b7913ff87"
MODEL_NAME = "doubao-1.5-thinking-vision-pro-250428"

# 创建方舟客户端
client = Ark(
    api_key=API_KEY,
    timeout=1800,
)

# 加载QA数据
qa_data_path = "../QA对/QA4.json"
with open(qa_data_path, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

    
# 加载混合检索结果
hybrid_results_path = "../outputs/outputs_4.0/混合检索/hybrid_results_k=50.json"
with open(hybrid_results_path, 'r', encoding='utf-8') as f:
    hybrid_results = json.load(f)


def calculate_similarity_metrics(predicted: str, reference: str) -> Dict[str, float]:
    """计算BLEU和ROUGE分数"""
    # 先对标准答案和生成答案进行jieba分词
    pred_tokens = list(jieba.cut(predicted))
    ref_tokens = list(jieba.cut(reference))
    
    # 在计算BLEU分数时添加平滑函数
    # 平滑函数（SmoothingFunction）用于在BLEU分数计算中避免因高阶n-gram（如2-gram、3-gram等）没有匹配而导致分数为0的情况。
    # 它通过对分母加1等方式，使得即使没有高阶匹配也能得到较小但非零的分数，从而更合理地反映生成文本与参考答案的相似度。
    # 高阶n-gram（如2-gram、3-gram等）是指由连续n个词组成的短语。例如，2-gram是连续两个词的组合，3-gram是连续三个词的组合。BLEU分数会统计这些n-gram在生成文本和参考答案中的重合情况，n越大，匹配难度越高，但能更好地反映语序和上下文的准确性。
    bleu_score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=SmoothingFunction().method1)
    
    # 计算ROUGE
    rouge = Rouge()
    rouge_scores = rouge.get_scores(predicted, reference)[0]
    
    return {
        "bleu": bleu_score,#关注精确匹配和词序，可能忽略同义词
        "rouge-1": rouge_scores["rouge-1"]["f"],#关注词汇覆盖（单个词的匹配程度）
        "rouge-2": rouge_scores["rouge-2"]["f"],#关注词语准确性（两个词的匹配程度）
        "rouge-l": rouge_scores["rouge-l"]["f"] #关注整体语义连贯性（最长公共子序列的匹配长度）
    }

# 加载检索结果
hybrid_results_path = "../outputs/outputs_4.0/混合检索/hybrid_results_k=50.json"
with open(hybrid_results_path, 'r', encoding='utf-8') as f:
    hybrid_results = json.load(f)

def evaluate_qa_chain(qa_data: List[Dict], hybrid_results: List[Dict]) -> Dict:
    """评估问答链性能"""
    results = []
    total_metrics = {
        "bleu": 0.0,
        "rouge-1": 0.0,
        "rouge-2": 0.0,
        "rouge-l": 0.0
    }
    
    # 创建问题到检索结果的映射
    results_map = {item["query"]: item for item in hybrid_results["results"]}
    
    for item in qa_data:
        try:
            # 获取该问题的检索结果
            query_result = results_map.get(item["question"])
            if not query_result:
                print(f"未找到问题 '{item['question']}' 的检索结果")
                continue
                
            # 构建提示词
            context = "\n".join([doc["source_content"] for doc in query_result["retrieved_documents"]])
            prompt = f"""你是一名金融保险领域的专家，擅长根据所获取的信息片段，对问题进行分析和推理。你的任务是根据所获取的上下文信息回答问题。
回答保持简洁，不必重复问题，不要添加与问题无关的任何内容。
如果上下文信息不足以回答问题，请明确说明。
如果上下文信息中包含多个片段，请综合考虑所有片段的信息来回答问题。
如果所提供的上下文中存在矛盾的信息，请指出并解释原因。
如果上下文信息中存在多余的信息，如符号、标点、空格等，请先判断是否可以忽略，确保最终输出结果清晰易懂，是流畅的内容。
如果上下文信息中存在不必要的细节或背景信息，请忽略这些信息。
如果上下文信息中存在繁体中文，请将其转换为简体字。
如果上下文信息中存在拼音，请将其转换为中文。
如果上下文信息中存在英文，请将其翻译为中文。
请根据以下上下文回答问题。
上下文：{context}
问题：{item["question"]}
请给出详细准确的回答："""
            
            # 调用API生成回答
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "你是一个专业的保险问题回答助手。"},
                    {"role": "user", "content": prompt}
                ]
            )
            predicted_answer = response.choices[0].message.content
            
            # 计算相似度指标
            metrics = calculate_similarity_metrics(predicted_answer, item["answer"])
            
            # 更新总分
            for metric in total_metrics:
                total_metrics[metric] += metrics[metric]
            
            results.append({
                "question": item["question"],
                "predicted_answer": predicted_answer,
                "reference_answer": item["answer"],
                "metrics": metrics,
                "retrieved_documents": query_result["retrieved_documents"]  # 保存检索到的文档
            })
            
            print(f"已处理 {len(results)}/{len(qa_data)} 个问题")
            
        except Exception as e:
            print(f"处理问题时出错: {str(e)}")
            continue
    
    # 计算平均分数
    num_questions = len(results)
    avg_metrics = {
        metric: total_metrics[metric] / num_questions 
        for metric in total_metrics
    }
    
    # 计算BLEU和ROUGE的平均值
    bleu_avg = avg_metrics["bleu"]
    rouge_avg = (avg_metrics["rouge-1"] + avg_metrics["rouge-2"] + avg_metrics["rouge-l"]) / 3
    
    return {
        "bleu_average": bleu_avg,
        "rouge_average": rouge_avg,
        "average_metrics": avg_metrics,
        "results": results
    }


# 执行评估
print("开始评估问答链性能...")
qa_results = evaluate_qa_chain(qa_data, hybrid_results)  # 使用 hybrid_results 替代 retriever

# 输出平均分数
print("\n问答链评估结果:")
print(f"BLEU: {qa_results['average_metrics']['bleu']:.4f}")
print(f"ROUGE-1: {qa_results['average_metrics']['rouge-1']:.4f}")
print(f"ROUGE-2: {qa_results['average_metrics']['rouge-2']:.4f}")
print(f"ROUGE-L: {qa_results['average_metrics']['rouge-l']:.4f}")

# 保存结果
output_dir = os.path.join("..", "outputs", "outputs_4.0", "问答链")
os.makedirs(output_dir, exist_ok=True)
output_file_path = os.path.join(output_dir, f'qa_chain_results_k=50.json')
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(qa_results, f, ensure_ascii=False, indent=2)

print(f"\n评估完成！结果已保存到: {output_file_path}")